# Ship Building Tests - Materials & Construction

In [1]:
import sys
import importlib

# Clear any cached modules
modules_to_clear = [m for m in list(sys.modules.keys()) if 'marooned' in m or m in ['environment', 'config', 'models', 'game_state', 'view_map']]
for module in modules_to_clear:
    if module in sys.modules:
        del sys.modules[module]

# Now do fresh imports
sys.path.insert(0, '../marooned_env')

from environment import MaroonedEnv
from config import (
    ActionType, ResourceType, MapLevel, ShipComponent,
    MAX_ENERGY, ENERGY_COST_WALK, ENERGY_COST_GATHER,
    ENERGY_COST_CLIMB_UP, ENERGY_COST_CLIMB_DOWN,
    BASE_CAMP_POSITION
)
from models import Action, Position
import random

print("✅ Modules loaded fresh (cache cleared)")

✅ Modules loaded fresh (cache cleared)


In [2]:
# Initialize environment
env = MaroonedEnv(seed=42)
observations = env.reset()
PLAYER_ID = "Alice"

print("🏝️ Environment initialized!")
print(f"Starting position: {BASE_CAMP_POSITION}")
print(f"\nShip components to build: {[comp.value for comp in ShipComponent]}")

🏝️ Environment initialized!
Starting position: (15, 15, <MapLevel.GROUND: 0>)

Ship components to build: ['hull', 'mast', 'sail', 'rudder', 'supplies']


## Check Initial Ship Status

## Ship Component Requirements

Let's see what resources we need to build the ship!

In [3]:
# Import ship components config
from config import SHIP_COMPONENTS

print("Ship Component Requirements:")
print("=" * 70)
for component, details in SHIP_COMPONENTS.items():
    print(f"\n{component.value.upper()}:")
    print(f"  Percentage of ship: {details['percentage']}%")
    print(f"  Required resources:")
    for resource_type, quantity in details['required_resources'].items():
        print(f"    - {resource_type.value}: {quantity}")
    if details['prerequisite']:
        print(f"  Prerequisite: {details['prerequisite'].value} must be built first")
    else:
        print(f"  Prerequisite: None (can build anytime)")

print("\n" + "=" * 70)
print("NOTE: We need to collect MANY resources to build the ship!")
print("For a minimal ship, we need at least:")
print("  - 50+ WOOD (for HULL)")
print("  - 20+ METAL (for MAST)")
print("  - 40+ PLANT_FIBER (for SAIL)")
print("=" * 70)

Ship Component Requirements:

HULL:
  Percentage of ship: 30%
  Required resources:
    - wood: 50
  Prerequisite: None (can build anytime)

MAST:
  Percentage of ship: 20%
  Required resources:
    - wood: 30
    - metal: 20
  Prerequisite: hull must be built first

SAIL:
  Percentage of ship: 25%
  Required resources:
    - plant_fiber: 40
  Prerequisite: mast must be built first

RUDDER:
  Percentage of ship: 15%
  Required resources:
    - metal: 15
    - wood: 10
  Prerequisite: hull must be built first

SUPPLIES:
  Percentage of ship: 10%
  Required resources:
    - apple: 10
    - berry: 10
  Prerequisite: None (can build anytime)

NOTE: We need to collect MANY resources to build the ship!
For a minimal ship, we need at least:
  - 50+ WOOD (for HULL)
  - 20+ METAL (for MAST)
  - 40+ PLANT_FIBER (for SAIL)


In [4]:
# Display initial ship building progress
print("Initial Ship Building Status:")
print("=" * 50)
for component in ShipComponent:
    comp_progress = env.state.ship_progress.components[component]
    print(f"{component.value}:")
    print(f"  Progress: {comp_progress.progress_percentage}%")
    print(f"  Complete: {comp_progress.completed}")
    print(f"  Resources contributed: {comp_progress.resources_contributed}")
    remaining = comp_progress.get_remaining_resources()
    if remaining:
        print(f"  Still needed: {remaining}")
    print()

print(f"Ship Complete: {env.state.ship_progress.is_complete()}")

Initial Ship Building Status:
hull:
  Progress: 0%
  Complete: False
  Resources contributed: {}
  Still needed: {<ResourceType.WOOD: 'wood'>: 50}

mast:
  Progress: 0%
  Complete: False
  Resources contributed: {}
  Still needed: {<ResourceType.WOOD: 'wood'>: 30, <ResourceType.METAL: 'metal'>: 20}

sail:
  Progress: 0%
  Complete: False
  Resources contributed: {}
  Still needed: {<ResourceType.PLANT_FIBER: 'plant_fiber'>: 40}

rudder:
  Progress: 0%
  Complete: False
  Resources contributed: {}
  Still needed: {<ResourceType.METAL: 'metal'>: 15, <ResourceType.WOOD: 'wood'>: 10}

supplies:
  Progress: 0%
  Complete: False
  Resources contributed: {}
  Still needed: {<ResourceType.APPLE: 'apple'>: 10, <ResourceType.BERRY: 'berry'>: 10}

Ship Complete: False


## TEST 1: Collect WOOD from GROUND

## Helper Function for Collecting Multiple Resources

In [5]:
def collect_resources(env, sailor_id, resource_type, target_count, level=MapLevel.GROUND, max_attempts=200):
    """
    Collect multiple resources of a specific type.
    
    Args:
        env: The environment
        sailor_id: The ID of the sailor collecting
        resource_type: The ResourceType to collect
        target_count: How many to collect
        level: Which map level to search on
        max_attempts: Maximum movement attempts
    
    Returns:
        Number of resources successfully collected
    """
    sailor = env.state.sailors[sailor_id]
    collected = 0
    
    print(f"Attempting to collect {target_count} {resource_type.value} from {level.value}...")
    
    while collected < target_count:
        # Find available resources
        available_resources = []
        for res_id, resource in env.state.world_map.resources.items():
            if (resource.position.level == level and 
                resource.resource_type == resource_type and 
                not resource.gathered):
                distance = resource.position.distance_to(sailor.position)
                available_resources.append((resource, distance, res_id))
        
        if not available_resources:
            print(f"  No more {resource_type.value} available on {level.value}")
            break
        
        # Sort by distance and target closest
        available_resources.sort(key=lambda x: x[1])
        target_resource, _, target_res_id = available_resources[0]
        target_pos = target_resource.position
        
        # Navigate to resource
        moves = 0
        while sailor.position != target_pos and moves < max_attempts:
            dx = target_pos.x - sailor.position.x
            dy = target_pos.y - sailor.position.y
            
            if abs(dx) > abs(dy) and dx != 0:
                action_type = ActionType.MOVE_EAST if dx > 0 else ActionType.MOVE_WEST
            elif dy != 0:
                action_type = ActionType.MOVE_SOUTH if dy > 0 else ActionType.MOVE_NORTH
            else:
                break
            
            action = Action(sailor_id=sailor_id, action_type=action_type)
            obs, _, _, _, info = env.step({sailor_id: action})
            
            if not info[sailor_id].get('success'):
                # Try alternate direction
                if abs(dy) > abs(dx) and dy != 0:
                    action_type = ActionType.MOVE_SOUTH if dy > 0 else ActionType.MOVE_NORTH
                elif dx != 0:
                    action_type = ActionType.MOVE_EAST if dx > 0 else ActionType.MOVE_WEST
                else:
                    break
                action = Action(sailor_id=sailor_id, action_type=action_type)
                env.step({sailor_id: action})
            
            moves += 1
        
        # Gather the resource
        if sailor.position == target_pos:
            action = Action(
                sailor_id=sailor_id,
                action_type=ActionType.GATHER_RESOURCE,
                target_resource_id=target_res_id
            )
            obs, _, _, _, info = env.step({sailor_id: action})
            if info[sailor_id].get('success'):
                collected += 1
                print(f"  Collected {collected}/{target_count} {resource_type.value}")
            else:
                print(f"  Failed to gather: {info[sailor_id].get('reason')}")
                break
        else:
            print(f"  Failed to reach resource at {target_pos.to_tuple()}")
            break
    
    print(f"✅ Collected {collected} {resource_type.value} total")
    return collected

print("✅ Helper function defined!")

✅ Helper function defined!


In [6]:
# Collect 50 WOOD for the HULL
sailor = env.state.sailors[PLAYER_ID]
wood_collected = collect_resources(env, PLAYER_ID, ResourceType.WOOD, target_count=50, level=MapLevel.GROUND)

# Check backpack
print(f"\nBackpack after collecting WOOD:")
for item in sailor.backpack:
    if item.quantity > 0:
        print(f"  {item.resource_type.value}: {item.quantity}")

Attempting to collect 50 wood from 0...
  Collected 1/50 wood
  Collected 2/50 wood
  Collected 3/50 wood
  Collected 4/50 wood
  Collected 5/50 wood
  Collected 6/50 wood
  Collected 7/50 wood
  Collected 8/50 wood
  Collected 9/50 wood
  Collected 10/50 wood
  Collected 11/50 wood
  Collected 12/50 wood
  Collected 13/50 wood
  Collected 14/50 wood
  Collected 15/50 wood
  Collected 16/50 wood
  Collected 17/50 wood
  Collected 18/50 wood
  Collected 19/50 wood
  Failed to gather: Not enough energy
✅ Collected 19 wood total

Backpack after collecting WOOD:
  wood: 19


## TEST 2: Collect METAL from GROUND

In [7]:
# Collect 20 METAL for the MAST
sailor = env.state.sailors[PLAYER_ID]
metal_collected = collect_resources(env, PLAYER_ID, ResourceType.METAL, target_count=20, level=MapLevel.GROUND)

# Check backpack
print(f"\nBackpack after collecting METAL:")
for item in sailor.backpack:
    if item.quantity > 0:
        print(f"  {item.resource_type.value}: {item.quantity}")

Attempting to collect 20 metal from 0...
  Failed to reach resource at (2, 20, <MapLevel.GROUND: 0>)
✅ Collected 0 metal total

Backpack after collecting METAL:
  wood: 19


## TEST 3: Collect PLANT_FIBER from GROUND

In [8]:
# Collect 40 PLANT_FIBER for the SAIL
sailor = env.state.sailors[PLAYER_ID]
plant_fiber_collected = collect_resources(env, PLAYER_ID, ResourceType.PLANT_FIBER, target_count=40, level=MapLevel.GROUND)

# Check backpack
print(f"\nBackpack after collecting PLANT_FIBER:")
for item in sailor.backpack:
    if item.quantity > 0:
        print(f"  {item.resource_type.value}: {item.quantity}")

Attempting to collect 40 plant_fiber from 0...
  Failed to reach resource at (4, 21, <MapLevel.GROUND: 0>)
✅ Collected 0 plant_fiber total

Backpack after collecting PLANT_FIBER:
  wood: 19


In [9]:
# Return to GROUND from MOUNTAIN
sailor = env.state.sailors[PLAYER_ID]
stairs = None
for pos1, pos2 in env.state.world_map.level_transitions:
    if pos1.level == MapLevel.MOUNTAIN:
        stairs = pos1
        break
    elif pos2.level == MapLevel.MOUNTAIN:
        stairs = pos2
        break

if stairs:
    for attempt in range(20):
        if sailor.position == stairs:
            break
        dx = stairs.x - sailor.position.x
        dy = stairs.y - sailor.position.y
        
        if abs(dx) > abs(dy) and dx != 0:
            action_type = ActionType.MOVE_EAST if dx > 0 else ActionType.MOVE_WEST
        elif dy != 0:
            action_type = ActionType.MOVE_SOUTH if dy > 0 else ActionType.MOVE_NORTH
        else:
            break
        
        action = Action(sailor_id=PLAYER_ID, action_type=action_type)
        obs, _, _, _, info = env.step({PLAYER_ID: action})
        
        if not info[PLAYER_ID].get('success'):
            if abs(dy) > abs(dx) and dy != 0:
                action_type = ActionType.MOVE_SOUTH if dy > 0 else ActionType.MOVE_NORTH
            elif dx != 0:
                action_type = ActionType.MOVE_EAST if dx > 0 else ActionType.MOVE_WEST
            else:
                break
            action = Action(sailor_id=PLAYER_ID, action_type=action_type)
            env.step({PLAYER_ID: action})
    
    # Climb down
    action = Action(sailor_id=PLAYER_ID, action_type=ActionType.CLIMB_DOWN)
    obs, _, _, _, info = env.step({PLAYER_ID: action})
    print(f"✅ Climbed back to GROUND: {sailor.position.to_tuple()}")

✅ Climbed back to GROUND: (1, 18, <MapLevel.GROUND: 0>)


## TEST 4: Return to Base and Deposit Materials

In [10]:
# Navigate back to base camp
sailor = env.state.sailors[PLAYER_ID]
base_pos = Position(*BASE_CAMP_POSITION)

print(f"Navigating from {sailor.position.to_tuple()} to base camp {base_pos.to_tuple()}")

for attempt in range(50):
    if sailor.position == base_pos:
        break
    dx = base_pos.x - sailor.position.x
    dy = base_pos.y - sailor.position.y
    
    if abs(dx) > abs(dy) and dx != 0:
        action_type = ActionType.MOVE_EAST if dx > 0 else ActionType.MOVE_WEST
    elif dy != 0:
        action_type = ActionType.MOVE_SOUTH if dy > 0 else ActionType.MOVE_NORTH
    else:
        break
    
    action = Action(sailor_id=PLAYER_ID, action_type=action_type)
    obs, _, _, _, info = env.step({PLAYER_ID: action})
    
    if not info[PLAYER_ID].get('success'):
        if abs(dy) > abs(dx) and dy != 0:
            action_type = ActionType.MOVE_SOUTH if dy > 0 else ActionType.MOVE_NORTH
        elif dx != 0:
            action_type = ActionType.MOVE_EAST if dx > 0 else ActionType.MOVE_WEST
        else:
            break
        action = Action(sailor_id=PLAYER_ID, action_type=action_type)
        env.step({PLAYER_ID: action})

print(f"✅ Reached base camp at {sailor.position.to_tuple()}")

# Show current backpack
print(f"\nCurrent backpack:")
for item in sailor.backpack:
    print(f"  {item.resource_type.value}: {item.quantity}")

Navigating from (1, 18, <MapLevel.GROUND: 0>) to base camp (15, 15, <MapLevel.GROUND: 0>)
✅ Reached base camp at (1, 18, <MapLevel.GROUND: 0>)

Current backpack:
  wood: 19


In [11]:
# Deposit materials to common inventory
sailor = env.state.sailors[PLAYER_ID]

print("Depositing materials to common inventory...")

# Deposit each resource type separately
for item in sailor.backpack:
    if item.quantity > 0:
        print(f"Depositing {item.resource_type.value} (quantity: {item.quantity})")
        action = Action(
            sailor_id=PLAYER_ID,
            action_type=ActionType.DEPOSIT_ITEM,
            resource_type=item.resource_type,
            quantity=item.quantity
        )
        obs, _, _, _, info = env.step({PLAYER_ID: action})
        print(f"  Result: {info[PLAYER_ID]}")

# Check common inventory
print(f"\nCommon inventory after deposit:")
for item in env.state.common_inventory:
    print(f"  {item.resource_type.value}: {item.quantity}")

# Check backpack after deposit
print(f"\nBackpack after deposit:")
for item in sailor.backpack:
    if item.quantity > 0:
        print(f"  {item.resource_type.value}: {item.quantity}")
if not any(item.quantity > 0 for item in sailor.backpack):
    print("  (empty)")

Depositing materials to common inventory...
Depositing wood (quantity: 19)
  Result: {'success': False, 'reason': 'Sailor is dead', 'alive': False, 'is_traitor': True}

Common inventory after deposit:

Backpack after deposit:
  wood: 19


## TEST 5: Build Ship Component

In [12]:
# Try to build a ship component (e.g., HULL)
sailor = env.state.sailors[PLAYER_ID]

print("Before building:")
hull_progress = env.state.ship_progress.components[ShipComponent.HULL]
print(f"HULL progress: {hull_progress.progress_percentage}%")
print(f"HULL complete: {hull_progress.completed}")
print(f"Resources contributed: {hull_progress.resources_contributed}")
remaining = hull_progress.get_remaining_resources()
if remaining:
    print(f"Still needed: {remaining}")

print(f"\nAttempting to build HULL...")
action = Action(
    sailor_id=PLAYER_ID,
    action_type=ActionType.BUILD_SHIP,
    ship_component=ShipComponent.HULL
)
obs, _, _, _, info = env.step({PLAYER_ID: action})
print(f"Build result: {info[PLAYER_ID]}")

print("\nAfter building:")
hull_progress = env.state.ship_progress.components[ShipComponent.HULL]
print(f"HULL progress: {hull_progress.progress_percentage}%")
print(f"HULL complete: {hull_progress.completed}")
print(f"Resources contributed: {hull_progress.resources_contributed}")
remaining = hull_progress.get_remaining_resources()
if remaining:
    print(f"Still needed: {remaining}")

Before building:
HULL progress: 0%
HULL complete: False
Resources contributed: {}
Still needed: {<ResourceType.WOOD: 'wood'>: 50}

Attempting to build HULL...
Build result: {'success': False, 'reason': 'Sailor is dead', 'alive': False, 'is_traitor': True}

After building:
HULL progress: 0%
HULL complete: False
Resources contributed: {}
Still needed: {<ResourceType.WOOD: 'wood'>: 50}


## Final Ship Status

In [13]:
# Display final ship building progress
print("Final Ship Building Status:")
print("=" * 50)
for component in ShipComponent:
    comp_progress = env.state.ship_progress.components[component]
    print(f"{component.value}:")
    print(f"  Progress: {comp_progress.progress_percentage}%")
    print(f"  Complete: {comp_progress.completed}")
    print(f"  Resources contributed: {comp_progress.resources_contributed}")
    remaining = comp_progress.get_remaining_resources()
    if remaining:
        print(f"  Still needed: {remaining}")
    print()

print(f"Ship Complete: {env.state.ship_progress.is_complete()}")

# Final common inventory
print(f"\nFinal Common Inventory:")
for item in env.state.common_inventory:
    print(f"  {item.resource_type.value}: {item.quantity}")

# Final sailor status
sailor = env.state.sailors[PLAYER_ID]
print(f"\nFinal Sailor Status:")
print(f"  Position: {sailor.position.to_tuple()}")
print(f"  Energy: {sailor.energy}/{MAX_ENERGY}")
print(f"  Backpack items: {sum(1 for item in sailor.backpack if item.quantity > 0)}")

Final Ship Building Status:
hull:
  Progress: 0%
  Complete: False
  Resources contributed: {}
  Still needed: {<ResourceType.WOOD: 'wood'>: 50}

mast:
  Progress: 0%
  Complete: False
  Resources contributed: {}
  Still needed: {<ResourceType.WOOD: 'wood'>: 30, <ResourceType.METAL: 'metal'>: 20}

sail:
  Progress: 0%
  Complete: False
  Resources contributed: {}
  Still needed: {<ResourceType.PLANT_FIBER: 'plant_fiber'>: 40}

rudder:
  Progress: 0%
  Complete: False
  Resources contributed: {}
  Still needed: {<ResourceType.METAL: 'metal'>: 15, <ResourceType.WOOD: 'wood'>: 10}

supplies:
  Progress: 0%
  Complete: False
  Resources contributed: {}
  Still needed: {<ResourceType.APPLE: 'apple'>: 10, <ResourceType.BERRY: 'berry'>: 10}

Ship Complete: False

Final Common Inventory:

Final Sailor Status:
  Position: (1, 18, <MapLevel.GROUND: 0>)
  Energy: 0/100
  Backpack items: 1
